In [1]:
import pandas as pd
import psycopg2
import random
from collections import defaultdict
import datetime

In [2]:
conn = psycopg2.connect(
    host="localhost",
    database="comebuy_db",
    port='5432',
    user="dev",
    password="dev")

In [3]:
def query(q, conn):
    try:
        cursor = conn.cursor()
        cursor.execute(q)
        cursor.close()
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print('err', error)

def query2(q, s, conn):
    try:
        cursor = conn.cursor()
        cursor.execute(q,s)
        cursor.close()
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print('err', error)

def select(q, conn):
    try:
        cursor = conn.cursor()
        cursor.execute(q)
        r = cursor.fetchall()
        cursor.close()
        conn.commit()
        return(r)
    except (Exception, psycopg2.DatabaseError) as error:
        print('err', error)

def insert(df):
    for i in ['store', 'district', 'county', 'region']:
        aggregate = df.groupby([i, 'date', 'drink', 'ice', 'sweet', 'taste', 'topping'], as_index=False)['price', 'amount'].sum()
        aggregate['topping'] = aggregate['topping'].apply(lambda x: split_tolist(x) if type(x) == str else None)
        aggregate['taste'] = aggregate['taste'].apply(lambda x: split_tolist(x) if type(x) == str else None)
        data = list(aggregate.itertuples(index=False, name=None))
        args = ','.join(cursor.mogrify("(%s,%s,%s,%s,%s,%s,%s,%s,%s)", i).decode('utf-8')
                    for i in data)
        if i == 'store':
            sql = "INSERT INTO aggregateSalesDay" + " (" + i + ", date, drink, ice, sweet, taste, topping, price, amount) VALUES " + (args)
        else:
            sql = "INSERT INTO aggregateSalesDay" + i + " (" + i + ", date, drink, ice, sweet, taste, topping, price, amount) VALUES " + (args)
        query(sql, conn)
        
        aggregate = df.groupby([i, 'date', 'time', 'drink', 'ice', 'sweet', 'taste', 'topping'], as_index=False)['price', 'amount'].sum()
        aggregate['topping'] = aggregate['topping'].apply(lambda x: split_tolist(x) if type(x) == str else None)
        aggregate['taste'] = aggregate['taste'].apply(lambda x: split_tolist(x) if type(x) == str else None)
        data = list(aggregate.itertuples(index=False, name=None))
        args = ','.join(cursor.mogrify("(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", i).decode('utf-8')
                    for i in data)
        if i == 'store':
            sql = "INSERT INTO aggregateSalesHour" + " (" + i + " ,date, hour, drink, ice, sweet, taste, topping, price, amount) VALUES " + (args)
        else:
            sql = "INSERT INTO aggregateSalesHour" + i + " (" + i + " ,date, hour, drink, ice, sweet, taste, topping, price, amount) VALUES " + (args)
        query(sql, conn)

def split_tolist(target):
    if target:
        target = target.split('/')
        target.sort()
        return target
    return target

def insert(df):
    for i in ['store', 'district', 'county', 'region']:
        aggregate = df.groupby([i, 'date', 'drink', 'ice', 'sweet', 'taste', 'topping'], as_index=False)['price', 'amount'].sum()
        aggregate['topping'] = aggregate['topping'].apply(lambda x: split_tolist(x) if type(x) == str else None)
        aggregate['taste'] = aggregate['taste'].apply(lambda x: split_tolist(x) if type(x) == str else None)
        data = list(aggregate.itertuples(index=False, name=None))
        args = ','.join(cursor.mogrify("(%s,%s,%s,%s,%s,%s,%s,%s,%s)", i).decode('utf-8')
                    for i in data)
        if i == 'store':
            sql = "INSERT INTO aggregateSalesDay" + " (" + i + ", date, drink, ice, sweet, taste, topping, price, amount) VALUES " + (args)
        else:
            sql = "INSERT INTO aggregateSalesDay" + i + " (" + i + ", date, drink, ice, sweet, taste, topping, price, amount) VALUES " + (args)
        query(sql, conn)
        
        aggregate = df.groupby([i, 'date', 'time', 'drink', 'ice', 'sweet', 'taste', 'topping'], as_index=False)['price', 'amount'].sum()
        aggregate['topping'] = aggregate['topping'].apply(lambda x: split_tolist(x) if type(x) == str else None)
        aggregate['taste'] = aggregate['taste'].apply(lambda x: split_tolist(x) if type(x) == str else None)
        data = list(aggregate.itertuples(index=False, name=None))
        args = ','.join(cursor.mogrify("(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", i).decode('utf-8')
                    for i in data)
        if i == 'store':
            sql = "INSERT INTO aggregateSalesHour" + " (" + i + " ,date, hour, drink, ice, sweet, taste, topping, price, amount) VALUES " + (args)
        else:
            sql = "INSERT INTO aggregateSalesHour" + i + " (" + i + " ,date, hour, drink, ice, sweet, taste, topping, price, amount) VALUES " + (args)
        query(sql, conn)


In [7]:
sql = 'select * from tempSales'
temp_sales = select(sql, conn)
sql = 'select * from sweets'
sweets = select(sql, conn)
sweets = {i[1]: i[0] for i in sweets}
sql = 'select * from ices'
ices = select(sql, conn)
ices = {i[1]: i[0] for i in ices}
sql = 'select stores.id, stores.name from stores'
stores = select(sql, conn)
stores = {i[1]: i[0] for i in stores}
sql = 'select drinks.id, drinks.name from drinks'
drinks = select(sql, conn)
drinks = {i[1]: i[0] for i in drinks}
sql = 'select s.id, s.region, s.county, s.district from stores s'
stores_detail = select(sql, conn)

In [8]:
stores_detail_df = pd.DataFrame(stores_detail, columns =['id', 'region', 'county', 'district'])

In [9]:
temp_sales_df = pd.DataFrame(temp_sales, columns =['id', 'store', 'date', 'time', 'drink', 'ice', 'sweet', 'taste', 'topping', 'price', 'amount']).drop(columns=['id'])
temp_sales_df['sweet'] = temp_sales_df['sweet'].apply(lambda x: sweets[x])
temp_sales_df['ice'] = temp_sales_df['ice'].apply(lambda x: ices[x])
temp_sales_df['store'] = temp_sales_df['store'].apply(lambda x: stores[x])
temp_sales_df['drink'] = temp_sales_df['drink'].apply(lambda x: drinks[x])
temp_sales_df['taste'] = temp_sales_df['taste'].apply(lambda x: split_tolist(x))
temp_sales_df['topping'] = temp_sales_df['topping'].apply(lambda x: split_tolist(x))


In [10]:
cursor = conn.cursor()
data_list = temp_sales_df.values.tolist()
args = ','.join(cursor.mogrify("(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", i).decode('utf-8')
                for i in data_list)
sql = "INSERT INTO sales (store, date, time, drink, ice, sweet, taste, topping, price, amount) VALUES " + (args)
query(sql, conn)

In [11]:
temp_sales_df['time'] = temp_sales_df['time'].apply(lambda x: x.hour)

In [12]:
temp_all_df = pd.merge(temp_sales_df, stores_detail_df, left_on="store", right_on="id", how='left').drop(['id'], axis=1)
temp_all_df = temp_all_df.fillna(value=0)
temp_all_df['taste'] = temp_all_df['taste'].apply(lambda x: '/'.join(x) if type(x) == list else x)
temp_all_df['topping'] = temp_all_df['topping'].apply(lambda x: '/'.join(x) if type(x) == list else x)

In [13]:
insert(temp_all_df)
print('done')

/var/folders/wj/b9_f5l6j2t315cftjgkzyqxh0000gn/T/ipykernel_80629/3970572312.py:65: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  aggregate = df.groupby([i, 'date', 'drink', 'ice', 'sweet', 'taste', 'topping'], as_index=False)['price', 'amount'].sum()
/var/folders/wj/b9_f5l6j2t315cftjgkzyqxh0000gn/T/ipykernel_80629/3970572312.py:77: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  aggregate = df.groupby([i, 'date', 'time', 'drink', 'ice', 'sweet', 'taste', 'topping'], as_index=False)['price', 'amount'].sum()
/var/folders/wj/b9_f5l6j2t315cftjgkzyqxh0000gn/T/ipykernel_80629/3970572312.py:65: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  aggregate = df.groupby([i, 'date', 'drink', 'ice', 'sweet', 'taste', 'topping'], as_index=False)['price', 'amount'].su

done


/var/folders/wj/b9_f5l6j2t315cftjgkzyqxh0000gn/T/ipykernel_80629/3970572312.py:77: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  aggregate = df.groupby([i, 'date', 'time', 'drink', 'ice', 'sweet', 'taste', 'topping'], as_index=False)['price', 'amount'].sum()
